HMM-DDM: Mouse Data
========================
In this notebook we will analyse a dataset of mice performing a two-alternative forced choice (TAFC) task developed in Kane, Senne, and Scott, 2025. mice performed this task daily in two-hour periods.

In [ ]:
using HiddenMarkovModels
using DriftDiffusionModels
using Plots
using CSV
using DataFrames
using Dates
using Random
using Distributions
using LinearAlgebra
using StatsBase
using JLD2
using Printf

## Load and Preprocess the data

In [ ]:
ddir = "../data/mouse_df.csv"
# load in the data
df = CSV.File(ddir) |> DataFrame;

In [ ]:
# Group by animal name and count trials
trial_counts = combine(groupby(df, :name), nrow => :trial_count)

# Sort by count in descending order
sort!(trial_counts, :trial_count, rev=true)

# pick a mouse of interest (start with mouse of most trials)
moi = trial_counts[2, :name]

# get the data for the mouse of interest
mouse_df = df[df.name .== moi, :]

# Filter out trials with "omission" outcome
valid_trials = findall(outcome -> outcome != "omission", mouse_df.outcome)
filtered_df = mouse_df[valid_trials, :]

# Map correct -> 1 and incorrect -> -1 (only for error and correct, omissions are gone)
numeric_outcomes = [outcome == "correct" ? 1 : -1 for outcome in filtered_df.outcome]

# Get reaction times, filter out "NAN" values
valid_rt_indices = findall(rt -> uppercase(string(rt)) != "NAN", filtered_df.rt)

# Apply both filters to keep data aligned
final_df = filtered_df[valid_rt_indices, :]
final_outcomes = numeric_outcomes[valid_rt_indices]

# Convert RTs to Float64
# final_rts = [parse(Float64, rt) for rt in final_df.rt]
final_rts = final_df.rt

# Extract just the date part from the timestamp strings
dates = [Date(split(dt)[1]) for dt in final_df.trial_datetime]

# Get unique dates in chronological order
unique_dates = sort(unique(dates))

# Create a vector of vectors, where each inner vector contains DDMResults for one day
results_by_date = Vector{Vector{DDMResult}}()

for date in unique_dates
    # Get indices for this date
    day_indices = findall(dates .== date)
    
    # Skip days with no valid data
    if isempty(day_indices)
        continue
    end
    
    # Extract RTs and outcomes for this date
    day_rts = final_rts[day_indices]
    day_outcomes = final_outcomes[day_indices]
    
    # Create DDMResult objects for this day
    day_results = [DDMResult(rt, choice) for (rt, choice) in zip(day_rts, day_outcomes)]
    
    # Add to our vector of vectors
    push!(results_by_date, day_results)
end

# Now calculate the sequence ends (cumulative sum of lengths)
seq_ends = cumsum([length(seq) for seq in results_by_date])

# Concatenate all results into a single vector
all_results = reduce(vcat, results_by_date)

# Visualize the correct and error RT distirbutions

In [ ]:
correct_trials = [x for x in all_results if x.choice == 1]
incorrect_trials = [x for x in all_results if x.choice == -1];

In [ ]:
rt_dist_plot = histogram([x.rt for x in correct_trials], bins=100, alpha=0.5, label="Correct", legend=:topright)
incorrect_rts = [x.rt for x in incorrect_trials]
h = fit(Histogram, incorrect_rts, range(0, maximum(incorrect_rts), length=101))
bar!(h.edges[1][1:end-1], -h.weights, width=step(h.edges[1]), 
     alpha=0.5, 
     label="Incorrect")
# Adjust y-axis to make room for the downward bars
xlabel!("RT (s)")
ylabel!("Count")
title!("RT Distribution for Mouse: $moi")
ylims!(minimum([-h.weights; 0]) * 1.1, maximum(h.weights) * 2)
xlims!(0, 2)

## Two State Model

In [ ]:
# generate a HMM to start from
trans_guess = [0.95 0.025 0.025; 0.025 0.95 0.025; 0.025 0.025 0.95]
init_guess = [0.99, 0.01, 0.00]

ddm_1_guess = DriftDiffusionModel(2.0, 0.5, 0.5, 0.3, 1)
ddm_2_guess = DriftDiffusionModel(0.7, 0.1, 0.5, 0.3, 1)
ddm_3_guess = DriftDiffusionModel(1.5, 0.25, 0.5, 0.3, 1)

dists_guess = [ddm_1_guess, ddm_2_guess, ddm_3_guess]

# create a prior distirbution for the initial states and transition matrix (sticky HMM)
α₀ = ones(3)
αₜ = ones(3, 3) # .* 2
αₜ[diagind(αₜ)] .= 3

hmm = PriorHMM(init_guess, trans_guess, dists_guess, αₜ, α₀)

hmm_est, lls = baum_welch(hmm, all_results; seq_ends=seq_ends)

In [ ]:
function state_dependent_histograms(true_data::Tuple{Vector{DDMResult}, Vector{Int}},
                                    simulated_data::Tuple{Vector{DDMResult}, Vector{Int}},
                                    n_bins::Int=100)

    default(guidefontsize=16, tickfontsize=14, legendfontsize=12)

    true_results, true_states = true_data
    sim_results, sim_states = simulated_data

    n_states = maximum(vcat(true_states, sim_states))  # assume states ∈ 1:n
    all_rts = [r.rt for r in true_results] ∪ [r.rt for r in sim_results]
    min_rt, max_rt = extrema(all_rts)
    bin_edges = range(min_rt, max_rt, length=n_bins+1)
    bin_width = (max_rt - min_rt) / n_bins
    bin_centers = range(min_rt + bin_width/2, max_rt - bin_width/2, length=n_bins)

    # Utility function
    function compute_hist_heights(data::Vector{Float64})
        if isempty(data)
            return zeros(n_bins)
        end
        hist = fit(Histogram, data, bin_edges)
        normalize(hist.weights ./ bin_width, 1)
    end

    # Build plots for each state
    plots = Vector{Plots.Plot}(undef, n_states)

    for k in 1:n_states
        # Extract trial indices for state k
        true_idx = findall(x -> x == k, true_states)
        sim_idx  = findall(x -> x == k, sim_states)

        true_correct = [r.rt for (i, r) in enumerate(true_results) if i in true_idx && r.choice == 1]
        true_error   = [r.rt for (i, r) in enumerate(true_results) if i in true_idx && r.choice == -1]
        sim_correct  = [r.rt for (i, r) in enumerate(sim_results) if i in sim_idx  && r.choice == 1]
        sim_error    = [r.rt for (i, r) in enumerate(sim_results) if i in sim_idx  && r.choice == -1]

        true_correct_heights = compute_hist_heights(true_correct)
        true_error_heights   = compute_hist_heights(true_error)
        sim_correct_heights  = compute_hist_heights(sim_correct)
        sim_error_heights    = compute_hist_heights(sim_error)

        p = plot(title="State $k Response Times", xlabel="RT (s)", ylabel="Density")

        # Positive: correct
        bar!(p, bin_centers, true_correct_heights,
            label="True (Correct)", alpha=0.8, color=:blue, bar_width=bin_width)
        bar!(p, bin_centers, sim_correct_heights,
            label="Simulated (Correct)", alpha=0.8, color=:lightblue, bar_width=bin_width)

        # Negative: error
        bar!(p, bin_centers, -true_error_heights,
            label="True (Error)", alpha=0.8, color=:red, bar_width=bin_width)
        bar!(p, bin_centers, -sim_error_heights,
            label="Simulated (Error)", alpha=0.8, color=:pink, bar_width=bin_width)

        hline!(p, [0], color=:black, linestyle=:dash, label=nothing)

        # Symmetric ticks
        max_y = maximum(abs.([
            maximum(true_correct_heights), maximum(true_error_heights),
            maximum(sim_correct_heights),  maximum(sim_error_heights)
        ]))
        y_ticks = collect(range(-max_y, max_y, length=7))
        yticks!(p, y_ticks, [@sprintf("%.2f", abs(y)) for y in y_ticks])

        plots[k] = p
    end

    return plot(plots..., layout=(n_states, 1), size=(800, 300 * n_states), legend=:topright)

end

In [ ]:
state_dependent_histograms(
    (all_results, most_likely_states),
    (dat_sim, seq_sim),
    100
)

In [ ]:
savefig("state_dependent_histograms.svg")

In [ ]:
posterior_plot = plot()
plot!(posterior[1, 1:1000], label="State 1", linewidth=2.0)
plot!(posterior[2, 1:1000], label="State 2", linewidth=2.0)
plot!(posterior[3, 1:1000], label="State 3", linewidth=2.0)
ylabel!("Pr(State i)")
xlabel!("Trial")

# savefig(posterior_plot, "posterior.svg")

In [ ]:
one_state_ddm = fit!(DriftDiffusionModel(v=1.1, B=3.3, τ=0.1), all_results)

In [ ]:
one_state_samples = [rand(one_state_ddm) for _ in 1:10000]
three_state_samples = rand(hmm_est, 10000)

In [ ]:
function ppc_asymmetric_histogram(true_data::Vector{DDMResult},
                                         sim1_data::Vector{DDMResult},
                                         sim_bestfit_data::Vector{DDMResult};
                                         n_bins::Int=100)

    default(guidefontsize=16, tickfontsize=14, legendfontsize=12)

    # Split correct/error
    split_choices(data) = (
        correct = [r.rt for r in data if r.choice == 1],
        error   = [r.rt for r in data if r.choice == -1]
    )

    true_c, true_e     = split_choices(true_data)
    sim1_c, sim1_e     = split_choices(sim1_data)
    bestfit_c, bestfit_e = split_choices(sim_bestfit_data)

    # Define shared bin edges
    all_rts = vcat(true_c, true_e, sim1_c, sim1_e, bestfit_c, bestfit_e)
    min_rt, max_rt = extrema(all_rts)
    bin_edges = range(min_rt, max_rt, length=n_bins+1)
    bin_width = (max_rt - min_rt) / n_bins
    bin_centers = range(min_rt + bin_width/2, max_rt - bin_width/2, length=n_bins)

    function compute_hist_heights(data::Vector{Float64})
        if isempty(data)
            return zeros(n_bins)
        end
        hist = fit(Histogram, data, bin_edges)
        hist.weights ./ (sum(hist.weights) * bin_width)
    end

    # Compute heights
    h = Dict(
        :true_c     => compute_hist_heights(true_c),
        :true_e     => compute_hist_heights(true_e),
        :sim1_c     => compute_hist_heights(sim1_c),
        :sim1_e     => compute_hist_heights(sim1_e),
        :bestfit_c  => compute_hist_heights(bestfit_c),
        :bestfit_e  => compute_hist_heights(bestfit_e),
    )

    # Max for symmetric y-axis
    max_y = maximum(abs.([
        h[:true_c]; h[:true_e];
        h[:sim1_c]; h[:sim1_e];
        h[:bestfit_c]; h[:bestfit_e]
    ]))
    y_ticks = range(-max_y, max_y, length=7)
    tick_labels = [@sprintf("%.2f", abs(y)) for y in y_ticks]

    # LEFT: 1-state vs true
    p1 = plot(title="True vs 1-State DDM", xlabel="RT (s)", ylabel="Density")
    bar!(p1, bin_centers, h[:true_c],     label="True (Correct)", color=:black, alpha=0.6, bar_width=bin_width)
    bar!(p1, bin_centers, h[:sim1_c],     label="1-State (Correct)", color=:orange, alpha=0.6, bar_width=bin_width)
    bar!(p1, bin_centers, -h[:true_e],    label="True (Error)", color=:black, alpha=0.6, bar_width=bin_width)
    bar!(p1, bin_centers, -h[:sim1_e],    label="1-State (Error)", color=:red, alpha=0.6, bar_width=bin_width)
    hline!(p1, [0], color=:black, linestyle=:dash, label=nothing)
    yticks!(p1, y_ticks, tick_labels)
    xlims!(0,4)

    # RIGHT: best-fit vs true
    p2 = plot(title="True vs Best-Fit Model", xlabel="RT (s)", ylabel="Density")
    bar!(p2, bin_centers, h[:true_c],      label="True (Correct)", color=:black, alpha=0.6, bar_width=bin_width)
    bar!(p2, bin_centers, h[:bestfit_c],   label="Best-Fit (Correct)", color=:blue, alpha=0.6, bar_width=bin_width)
    bar!(p2, bin_centers, -h[:true_e],     label="True (Error)", color=:black, alpha=0.6, bar_width=bin_width)
    bar!(p2, bin_centers, -h[:bestfit_e],  label="Best-Fit (Error)", color=:purple, alpha=0.6, bar_width=bin_width)
    hline!(p2, [0], color=:black, linestyle=:dash, label=nothing)
    yticks!(p2, y_ticks, tick_labels)
    xlims!(0,4)

    return plot(p1, p2, layout=(1,2), size=(1000, 400), legend=:topright)
end


In [ ]:
ppc = ppc_asymmetric_histogram(all_results,
one_state_samples,
three_state_samples[2])

savefig(ppc, "ppc_asymmetric_histogram.svg")

lls

In [ ]:
using StatsBase

function ks_comparison_plot(true_data::Vector{DDMResult},
                            sim1_data::Vector{DDMResult},
                            sim_bestfit_data::Vector{DDMResult})

    default(guidefontsize=16, tickfontsize=14, legendfontsize=12)

    # Extract sorted RTs for correct and error trials
    function split_sorted_rts(data)
        correct = sort([r.rt for r in data if r.choice == 1])
        error   = sort([r.rt for r in data if r.choice == -1])
        return correct, error
    end

    tc, te = split_sorted_rts(true_data)
    s1c, s1e = split_sorted_rts(sim1_data)
    bfc, bfe = split_sorted_rts(sim_bestfit_data)

    # Compute ECDF values
    function ecdf_xy(data)
        ec = ecdf(data)
        x = sort(data)
        y = ec.(x)
        return x, y
    end

    # === Correct Trials ===
    tc_x, tc_y = ecdf_xy(tc)
    s1c_x, s1c_y = ecdf_xy(s1c)
    bfc_x, bfc_y = ecdf_xy(bfc)

    p1 = plot(title="KS Plot – Correct Trials",
              xlabel="RT (s)", ylabel="ECDF", legend=:bottomright)

    plot!(p1, tc_x, tc_y, label="True", lw=2, color=:black)
    plot!(p1, s1c_x, s1c_y, label="1-State", lw=2, color=:gray)
    plot!(p1, bfc_x, bfc_y, label="3-State", lw=2, color=:blue)

    # === Error Trials ===
    te_x, te_y = ecdf_xy(te)
    s1e_x, s1e_y = ecdf_xy(s1e)
    bfe_x, bfe_y = ecdf_xy(bfe)

    p2 = plot(title="KS Plot – Error Trials",
              xlabel="RT (s)", ylabel="ECDF", legend=:bottomright)

    plot!(p2, te_x, te_y, label="True", lw=2, color=:black)
    plot!(p2, s1e_x, s1e_y, label="1-State", lw=2, color=:red)
    plot!(p2, bfe_x, bfe_y, label="3-State", lw=2, color=:purple)

    return plot(p1, p2, layout=(1,2), size=(1000, 400))
end


In [ ]:
ks = ks_comparison_plot(all_results, 
                   one_state_samples,
                   three_state_samples[2])

savefig(ks, "ks_plot.svg")

In [ ]:
sum(lls)

## Cross Validation
---
Determine if our two-state model or one-statemodel is superior.

In [ ]:
one_state_model = PriorHMM(
    ones(1),
    ones(1, 1),
    [DriftDiffusionModel(2.0, 0.5, 0.5, 0.5, 1)],
)

ddm, ddm_likelihood = baum_welch(one_state_model, all_results; seq_ends=seq_ends)

## Save Results: Optional

In [ ]:
@save joinpath("../data/results", moi* "_hmm.jld2") hmm_est

## Confirm the LL converges!

In [ ]:
plot(lls, title="Log-Likelihood", xlabel="Iteration", ylabel="ℓℓ", legend=false)

## Plot the Posterior Distribution of the States


In [ ]:
# get the posterior distribution of the hidden states
posterior = forward(hmm_est, all_results; seq_ends=seq_ends)
γ = posterior[1]

plot(γ[1, 1:1000], title="Posterior Probabilites", xlabel="Trial", ylabel="Probability of State i", label="State 1", linewidth=2)
plot!(γ[2, 1:1000], label="State 2", linewidth=2)

## Post-Hoc Analysis of HMM-DDM

In [ ]:
fitted_models = [load(joinpath("../data/results", mouse_name * "_hmm.jld2")) for mouse_name in trial_counts.name[1:3]]

In [ ]:
function plot_two_state_hmm_heatmap(state1_probs::Vector{Float64}, sequence_ends::Vector{Int}; 
                                  color_scheme=:viridis, 
                                  fig_size=(900, 400),
                                  title="Probability of State 1")
    # state1_probs is a vector of length N (total time points across all sequences)
    # containing probabilities of being in State 1
    
    N = length(state1_probs)
    
    # Calculate sequence starts
    sequence_starts = [1; sequence_ends[1:end-1] .+ 1]
    n_sequences = length(sequence_ends)
    
    # Find maximum sequence length
    max_length = maximum(sequence_ends .- sequence_starts .+ 1)
    
    # Create data array for the heatmap (initialize with NaN)
    heatmap_data = fill(NaN, (n_sequences, max_length))
    
    # Fill in data for each sequence
    for seq in 1:n_sequences
        start_idx = sequence_starts[seq]
        end_idx = sequence_ends[seq]
        seq_length = end_idx - start_idx + 1
        
        # Extract probabilities for this sequence
        heatmap_data[seq, 1:seq_length] = state1_probs[start_idx:end_idx]
    end
    
    # Create the heatmap
    hm = heatmap(heatmap_data,
                c=color_scheme,
                title=title,
                xlabel="Time Step",
                ylabel="Sequence",
                yticks=(1:n_sequences, ["Seq $i" for i in 1:n_sequences]),
                clim=(0,1),
                colorbar_title="Probability",
                fillalpha=0.8,
                size=fig_size)
    
    return hm
end

In [ ]:
function state_dependent_histogram(true_data::Tuple{Vector{DDMResult}, Vector{Int}},
                                 simulated_data::Tuple{Vector{DDMResult}, Vector{Int}},
                                 n_bins::Int=100)
    # Unpack the data
    true_results, true_states = true_data
    sim_results, sim_states = simulated_data
    
    # Create empty plots
    p1 = plot(title="State 1 Response Times", xlabel="RT (s)", ylabel="Density")
    p2 = plot(title="State 2 Response Times", xlabel="RT (s)", ylabel="Density")
    
    # Stratify data by state and correctness
    # State 1
    state1_true_idx = findall(x -> x == 1, true_states)
    state1_sim_idx = findall(x -> x == 1, sim_states)
    
    state1_true_correct = [r.rt for (i, r) in enumerate(true_results) if i in state1_true_idx && r.choice == 1]
    state1_true_error = [r.rt for (i, r) in enumerate(true_results) if i in state1_true_idx && r.choice == -1]
    
    state1_sim_correct = [r.rt for (i, r) in enumerate(sim_results) if i in state1_sim_idx && r.choice == 1]
    state1_sim_error = [r.rt for (i, r) in enumerate(sim_results) if i in state1_sim_idx && r.choice == -1]
    
    # State 2
    state2_true_idx = findall(x -> x == 2, true_states)
    state2_sim_idx = findall(x -> x == 2, sim_states)
    
    state2_true_correct = [r.rt for (i, r) in enumerate(true_results) if i in state2_true_idx && r.choice == 1]
    state2_true_error = [r.rt for (i, r) in enumerate(true_results) if i in state2_true_idx && r.choice == -1]
    
    state2_sim_correct = [r.rt for (i, r) in enumerate(sim_results) if i in state2_sim_idx && r.choice == 1]
    state2_sim_error = [r.rt for (i, r) in enumerate(sim_results) if i in state2_sim_idx && r.choice == -1]
    
    # Handle case where arrays might be empty
    if isempty(state1_true_correct) && isempty(state1_true_error) && 
       isempty(state1_sim_correct) && isempty(state1_sim_error) &&
       isempty(state2_true_correct) && isempty(state2_true_error) && 
       isempty(state2_sim_correct) && isempty(state2_sim_error)
        return plot(title="No data available")
    end
    
    # Find common RT range for appropriate binning
    all_rts = vcat(
        state1_true_correct, state1_true_error, state1_sim_correct, state1_sim_error,
        state2_true_correct, state2_true_error, state2_sim_correct, state2_sim_error
    )
    
    min_rt = minimum(all_rts)
    max_rt = maximum(all_rts)
    rt_range = (min_rt, max_rt)
    
    # Calculate histogram data manually
    # Create the bins
    bin_edges = range(min_rt, max_rt, length=n_bins+1)
    bin_width = (max_rt - min_rt) / n_bins
    bin_centers = range(min_rt + bin_width/2, max_rt - bin_width/2, length=n_bins)
    
    # Function to compute histogram heights
    function compute_hist_heights(data, normalize=true)
        if isempty(data)
            return zeros(n_bins)
        end
        hist = fit(Histogram, data, bin_edges)
        heights = hist.weights
        if normalize
            heights = heights ./ (sum(heights) * bin_width)
        end
        return heights
    end
    
    # Compute histogram heights for all datasets
    s1_true_correct_heights = compute_hist_heights(state1_true_correct)
    s1_true_error_heights = compute_hist_heights(state1_true_error)
    s1_sim_correct_heights = compute_hist_heights(state1_sim_correct)
    s1_sim_error_heights = compute_hist_heights(state1_sim_error)
    
    s2_true_correct_heights = compute_hist_heights(state2_true_correct)
    s2_true_error_heights = compute_hist_heights(state2_true_error)
    s2_sim_correct_heights = compute_hist_heights(state2_sim_correct)
    s2_sim_error_heights = compute_hist_heights(state2_sim_error)
    
    # Plot histograms using bar plots
    # State 1 - Correct trials (positive y-axis)
    bar!(p1, bin_centers, s1_true_correct_heights, 
        label="True Data (Correct)", alpha=0.8, color=:blue, 
        bar_width=bin_width, linewidth=0)
    
    bar!(p1, bin_centers, s1_sim_correct_heights, 
        label="Simulated Data (Correct)", alpha=0.8, color=:lightblue, 
        bar_width=bin_width, linewidth=0)
    
    # State 1 - Error trials (negative y-axis)
    bar!(p1, bin_centers, -s1_true_error_heights, 
        label="True Data (Error)", alpha=0.8, color=:red, 
        bar_width=bin_width, linewidth=0)
    
    bar!(p1, bin_centers, -s1_sim_error_heights, 
        label="Simulated Data (Error)", alpha=0.8, color=:pink, 
        bar_width=bin_width, linewidth=0)
    
    # State 2 - Correct trials (positive y-axis)
    bar!(p2, bin_centers, s2_true_correct_heights, 
        label="True Data (Correct)", alpha=0.8, color=:blue, 
        bar_width=bin_width, linewidth=0)
    
    bar!(p2, bin_centers, s2_sim_correct_heights, 
        label="Simulated Data (Correct)", alpha=0.8, color=:lightblue, 
        bar_width=bin_width, linewidth=0)
    
    # State 2 - Error trials (negative y-axis)
    bar!(p2, bin_centers, -s2_true_error_heights, 
        label="True Data (Error)", alpha=0.8, color=:red, 
        bar_width=bin_width, linewidth=0)
    
    bar!(p2, bin_centers, -s2_sim_error_heights, 
        label="Simulated Data (Error)", alpha=0.8, color=:pink, 
        bar_width=bin_width, linewidth=0)
    
    # Add horizontal line at y=0
    hline!(p1, [0], color=:black, linestyle=:dash, label=nothing)
    hline!(p2, [0], color=:black, linestyle=:dash, label=nothing)
    
    # Manually set y-ticks with absolute values as labels
    # Find the maximum y value (positive or negative) to set symmetric ticks
    max_y1 = max(
        maximum(s1_true_correct_heights), 
        maximum(s1_sim_correct_heights),
        maximum(s1_true_error_heights),
        maximum(s1_sim_error_heights)
    )
    
    max_y2 = max(
        maximum(s2_true_correct_heights), 
        maximum(s2_sim_correct_heights),
        maximum(s2_true_error_heights),
        maximum(s2_sim_error_heights)
    )
    
    # Create symmetric tick positions
    y_ticks1 = collect(range(-max_y1, max_y1, length=7))
    y_ticks2 = collect(range(-max_y2, max_y2, length=7))

    # Format tick labels with restricted significant figures (2 decimal places)
    format_tick(y) = @sprintf("%.2f", abs(y))

    # Set ticks with formatted absolute values as labels
    yticks!(p1, y_ticks1, [format_tick(y) for y in y_ticks1])
    yticks!(p2, y_ticks2, [format_tick(y) for y in y_ticks2])
    
    # Add annotations 
    max_correct_height1 = maximum(s1_true_correct_heights)
    max_error_height1 = maximum(s1_true_error_heights)
    
    max_correct_height2 = maximum(s2_true_correct_heights)
    max_error_height2 = maximum(s2_true_error_heights)
    
    # Create combined plot
    combined_plot = plot(p1, p2, layout=(2,1), size=(1100, 800), legend=:topright)
    
    return combined_plot
end